In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals
%matplotlib inline
from matplotlib import pyplot
import caffe2
import numpy as np
import os
import shutil
import operator
import caffe2.python.predictor.predictor_exporter as pe
import os
import cv2
import random

from caffe2.python import (
    brew,
    core,
    model_helper,
    net_drawer,
    optimizer,
    visualize,
    workspace,
)

# If you would like to see some really detailed initializations,
# you can change --caffe2_log_level=0 to --caffe2_log_level=-1
core.GlobalInit(['caffe2', '--caffe2_log_level=0'])
print("Necessities imported!")

Necessities imported!


In [2]:
def generateSequence(index,seqSize,noisyDatPath,refDatPath):
    noisySeq = []
    refSeq = []
    imgList = os.listdir(noisyDatPath)
    imgList.sort()
   
    if((index+seqSize)>len(imgList)):
        seqSize = len(imgList) - index
  
    for i in range(seqSize):
        noisyImg = cv2.imread(noisyDatPath+"//"+imgList[index+i])
        refImg   = cv2.imread(refDatPath+"//"+imgList[index+i])
        #noisyImg = cv2.imread(noisyDatPath+"//"+imgList[index+i])
        #refImg   = cv2.imread(refDatPath+"//"+imgList[index+i])
        noisySeq.append(noisyImg)
        refSeq.append(refImg)
    
        
    return noisySeq,refSeq
    
    

In [3]:
def generateSubseqence(srcNoisyImg,srcRefImg,seqSize,subSeqSize):
    
    noisySubSeq = []
    refSubSeq = []
    subSeqWindow = 32 #trying to create subimages of size 32x32, paper works with 128x128

    width,height = noisySeq[0].shape[0],noisySeq[0].shape[1]
    
    for k in range(subSeqSize):
        initW = random.randint(0,width-subSeqWindow-1)
        initH = random.randint(0,height-subSeqWindow-1)
        for i in range(seqSize):
            if(i>len(srcNoisyImg)-1):
                break
            subImageNoisy = srcNoisyImg[i][initW:initW+subSeqWindow,initH:initH+subSeqWindow,:3]
            subImageRef   = srcRefImg[i][initW:initW+subSeqWindow,initH:initH+subSeqWindow,:3]
            wN,hN = subImageRef.shape[0],subImageRef.shape[1]
            if((wN != subSeqWindow) or(hN != subSeqWindow)):
                print("width or height does not match:"+str(wN)+"x"+str(hN)+" initW:"+str(initW)+":"+str(initW+subSeqWindow)+" initH:"+str(initH)+":"+str(initH+subSeqWindow))
            noisySubSeq.append(subImageNoisy)
            refSubSeq.append(subImageRef)
        
    return noisySubSeq,refSubSeq
    

In [181]:
def showImage(img):
    pyplot.figure()
    pyplot.subplot(1,2,1)
    pyplot.imshow(img)
    pyplot.axis('on')
    pyplot.title('Original image = RGB')

In [164]:
CONV_KERNEL_SIZE  = 3
CONV_KERNEL_PADING = 1
def AddRecurrentBlock(model,data,layerNo):
    convR1 = brew.conv(model, data, 'conv1', dim_in=1, dim_out=20, kernel=5)
    
    

In [158]:
def AddEncoderDecoder(model, data):
    
    #ENCODER PART
    # Image size: 32 x 32 -> 32 x 32
    conv1 = AddRecurrentBlock(model,data,1)
    # Image size: 32 x 32 -> 16 x 16
    pool1 = brew.max_pool(model, conv1, 'pool1', kernel=2, stride=2)

    # Image size: 16 x 16 -> 16 x 16
    conv2 = AddRecurrentBlock(model,pool1,2)
    # Image size: 8x8 -> 8x8
    pool2 = brew.max_pool(model, conv2, 'pool2', kernel=2, stride=2)
    
    # Image size: 8x8 -> 8 x 8
    conv3 = AddRecurrentBlock(model,pool2,3)
    # Image size: 8x8 -> 4x4
    pool3 = brew.max_pool(model, conv3, 'pool3', kernel=2, stride=2)
    
    # Image size: 4x4 -> 4x4
    conv4 = AddRecurrentBlock(model,pool3,4)
    
    ###DECODER PART
    upconv3 = brew.conv_transpose(model, conv4, "upcon3", dim_in=101, dim_out=101, kernel=3, pad=1)
    upsample3 = brew.conv_transpose(model, upconv3, "upsample3", dim_in=101, dim_out=101, kernel=2, stride=2)
    
    upconv2 = brew.conv_transpose(model, upsample3, "upcon2", dim_in=101, dim_out=76, kernel=3, pad=1)
    upsample2 = brew.conv_transpose(model, upconv2, "upsample2", dim_in=76, dim_out=76, kernel=2, stride=2)
    
    upconv1 = brew.conv_transpose(model, conv2, "upcon1", dim_in=76, dim_out=1, kernel=3, pad=1)
    upsample1 = brew.conv_transpose(model, upconv1, "upsample1", dim_in=1, dim_out=1, kernel=2, stride=2)
    

In [184]:
noisyTraining = "//home//rohit//Rohit//Internship//caffe//projects//ML_TAA//data//trainingData//Noisy"
refTraining = "//home//rohit//Rohit//Internship//caffe//projects//ML_TAA//data//trainingData//Ref"
i = 0
if(len(os.listdir(noisyTraining)) !=  len(os.listdir(refTraining))):
    print("Error: Training and ref sizes match!")

dirSize = len(os.listdir(noisyTraining))
seqSize = 3
subSeqSize = 3

while(i < dirSize):
    noisySeq,refSeq = generateSequence(i,seqSize,noisyTraining,refTraining)
    noisySubSeq,refSubSeq = generateSubseqence(noisySeq,refSeq,seqSize,subSeqSize)
    
    '''if i == 0:
        for k in range(len(noisySubSeq)):
            showImage(noisySubSeq[k])
            showImage(refSubSeq[k])'''
    i+=seqSize


In [ ]:
while(i < dirSize):
    noisySeq,refSeq = generateSequence(i,seqSize,noisyTraining,refTraining);
    
    i+=seqSize